<a href="https://colab.research.google.com/github/eaphilli/clinical-notes-nlp/blob/main/parse_t2d_notes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Parse T2D Notes
# Summary of assignment

## **Current strategy:**
1. **Implement an LLM-based extractor** - I chose an LLM-based extractor rather than NER because I am more familiar with this technique, so would be faster to implement in the 2 hour limit, and there could be some complexities around natural language in clinical notes that the NER might find difficult to handle (e.g. inference of complications).
2. **Entities to extract** - I decided to focus on extracting complications related to T2D, since this seems like an important outcome measure for patients with T2D that wasn't always explicitly mentioned in all notes (which also reinforced my decision to go with an LLM approach rather than NER). However, because I chose the LLM approach, I found that the model accuracy declined the more entities I chose to put into the JSON output.
Therefore, it was an explicit tradeoff that I made to get higher accuracy on the more important outcome measure (complications) at the expense of extracting more entities (e.g. lifestyle factors, plans).
3. **Validation** - I use json-repair to fix any invalid JSON that the LLM produces. If the JSON is so borked that the json-repair library can't fix it, I ask the LLM to re-analyze the note (with a higher temperature). We'll repeat this 3 times, and if all attempts fail, we'll output an error JSON.
4. **Evaluation** - I mostly tried using model grading for evaluation. However, this approach doesn't seem to work very well with the current grading prompt. My prompt was not specific enough for the grader to identify


## **Future strategy / improvements:**
1. **Overall approach** - A heirarchical/multi-layered NER & LLM combination approach might be less error prone and more structured than either alone. For example, if we could extract the relevant entities using NER, then we could use an LLM to standardize the entities according to a certain coding strategy, yielding more structured outputs. Then another LLM layer could synthesize the structured NER outputs, extrapolating complications based on the pre-structured entities.
2. **Entities to extract** - One of the largest issues I faced was related to variable term formats (e.g. "PRN" vs "as needed", or A1c vs HbA1C, or symptoms described in different ways). One idea is to add a term normalization step (such that all terms are in LOINC or SNOMED codes, or in FHIR format) as part of either a follow-on step or a precursor step (ie. entity linking). This could be done using a RAG method that searches through FHIR/LOINC/SNOMED documentation and identifies the proper mapping for each entity.
3. **Validation** - I didn't evaluate how well repair_json worked, so I'm not positive it's the best option. Given more time I would've tried OpenAI's function calling or "JSON mode".
4. **Evaluation** - I mostly tried using model grading for evaluation. However, this approach doesn't seem to work very well with the current grading prompt (just via manual inspection). A way to get more accurate grades is to use a larger model to do the grading, or to hand curate ground truth labels for the test data and have a model compare the output vs the ground truth (I didn't want to do this out of time management). It also would've helped if I had been able to map key entities to the appropriate terminology. Otherwise it's difficult for the grader to know what is the right terminology.


In [109]:
!pip install langchain langchain_community openai pandas jsonschema json-repair

In [110]:
from google.colab import drive, userdata
from openai import OpenAI
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate
import pandas as pd
import json
from json_repair import repair_json
from jsonschema import validate, ValidationError
import os

OPENAI_API_KEY = userdata.get('OPENAI_API_KEY')
os.environ['OPENAI_API_KEY'] = OPENAI_API_KEY
client = OpenAI()

In [28]:
# Mount your Google Drive
drive.mount('/content/drive')

# Replace 'path/to/your/excel_file.xlsx' with the actual path to your Excel file in your Google Drive
excel_file_path = '/content/drive/My Drive/Clinical Notes/synthetic_clinical_notes.xlsx'

try:
  # Load the Excel file into a pandas DataFrame
  df = pd.read_excel(excel_file_path)

  # Now you can work with the DataFrame 'df'
  print(df.head()) # Print the first few rows

except FileNotFoundError:
  print(f"Error: File not found at {excel_file_path}")
except Exception as e:
  print(f"An error occurred: {e}")

df.head()

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
                                                note
0  Patient: John Smith, 58-year-old male\nMedical...
1  Patient: Linda Green, 45-year-old female\nMedi...
2  Patient: Michael Brown, 62-year-old male\nMedi...
3  Patient: Sarah Johnson, 50-year-old female\nMe...
4  Patient: Carlos Ramirez, 55-year-old male\nMed...


,note
0,"Patient: John Smith, 58-year-old male\nMedical..."
1,"Patient: Linda Green, 45-year-old female\nMedi..."
2,"Patient: Michael Brown, 62-year-old male\nMedi..."
3,"Patient: Sarah Johnson, 50-year-old female\nMe..."
4,"Patient: Carlos Ramirez, 55-year-old male\nMed..."


# Create prompt & Structure Output

In [151]:
def extract_json_response(client, prompt, retries=3):
    """
    Extracts a JSON response from an LLM (gpt-4) given a prompt using temperature=0
    to achieve as close to deterministic response as possible.
    If the response is invalid JSON, tries to repair using json-repair.
    If it cannot be repaired, reattempts the LLM call with higher temperature for a
    maximum of 3 times (can be changed by setting retries).
    """
    llm_low_temp = ChatOpenAI(model_name="gpt-4", temperature=0)
    response = llm_low_temp.invoke(prompt)

    # Ensure proper JSON parsing.
    structured_output = repair_json(response.content) # https://pypi.org/project/json-repair/ (fixes syntax, repairs arrays/objects, auto-completes missing values)
    if not structured_output: # If the string was super broken this will be an empty string
        print("Failed to repair JSON: attempt 0")
        llm_hight_temp = ChatOpenAI(model_name="gpt-4", temperature=0.5)
        for i in range(retries):
            new_response = llm_hight_temp.invoke(prompt)
            structured_output = repair_json(response.content)
            if not structured_output:
                print(f"Failed to repair JSON: attempt {i+1}")
                continue
            else:
                return structured_output
        return '{"error": "Malformed JSON produced by GPT-4 after 4 tries."}'

    return structured_output

def extract_structured_data_from_note(client, note):
    prompt_template = PromptTemplate.from_template(
        """
        You are an expert medical language model. Your goal is to extract and structure relevant clinical information from a clinical note about a patient with Type 2 Diabetes (T2D).
        Most importantly, your job is to identify when an individual may be experiencing complications related to T2D.
        Some complications of T2D are below, and are indicated by the following symptoms:
        - "Peripheral neuropathy": Indicated by pain, numbness, tingling, loss of feeling in the extremities.
        - "Diabetic nephropathy": Indicated by elevated creatinine levels, protein in urine (albuminuria), swelling in legs and feet, fatigue, high blood pressure.
        - "Diabetic retinopathy": Indicated by blurred vision, floaters, difficulty seeing at night, partial or total vision loss.
        - "Skin conditions": Indicated by dry skin, itching, frequent skin infections, slow wound healing, dark patches on skin (acanthosis nigricans).
        - "Heart and blood vessel disease": Indicated by high blood pressure, chest pain, shortness of breath, irregular heartbeat, swelling in legs, dizziness, increased risk of stroke or heart attack.

        Do **not** make up information that is not contained within the note, and only provide information that is relevant to Type 2 Diabetes or another clinical diagnosis.
        When inputting the display text, dosage, frequency, and units for medications or observations, adhere to standard FHIR conventions, nomenclature, and labels.
        Ensure that all entities are labeled according to the FHIR coding scheme.
        For example:
        - "A1C 7.2%" -> "Hemoglobin A1c 7.2%"
        - "blood pressure (BP) 120/80" -> "Systolic blood pressure 120 mmHg, Diastolic blood pressure 80 mmHg"
        - "HR/pulse 82" -> "Heart rate 82 beats/min"

        Here is the note:
        {note}


        Return the output in this JSON format **only**. If an item within a specific key (e.g. observations: glucose ) is not in the note, the value should be null (e.g. "observations": {{"glucose": null}}).
        {{
        "patient": {{
            "id": "<string>",
            "name": "<string>",
            "age": <integer>,
            "sex": "<string>"
        }},
        "symptoms": ["<string>"],
        "suspected_complications": ["<string>"],
        "comorbidities": ["<string>"],
        "medications": [
            {{ "display": "<string>", "dosage": "<string>", "frequency": "<string>" }}
        ],
        "observations": [
            {{ "display": "<string>", "value": <float>, "unit": "<string>" }},
        ]
        }}
        """
    )
    prompt = prompt_template.format(note=note)
    structured_output = extract_json_response(client, prompt)
    return structured_output


In [152]:
# Apply GPT Extraction & inspect manually
structured_data = [extract_structured_data_from_note(client, note) for note in df["note"]]
for i in range(5):
    print(json.dumps(json.loads(structured_data[i]), indent=4))


{
    "patient": {
        "id": "1234",
        "name": "John Smith",
        "age": 58,
        "sex": "male"
    },
    "symptoms": [
        "fatigue",
        "blurred vision",
        "mild numbness in feet"
    ],
    "suspected_complications": [
        "Peripheral neuropathy",
        "Diabetic retinopathy"
    ],
    "comorbidities": [
        "Type 2 Diabetes",
        "Hypertension"
    ],
    "medications": [
        {
            "display": "Metformin",
            "dosage": "500 mg",
            "frequency": "BID"
        },
        {
            "display": "Insulin",
            "dosage": "occasional",
            "frequency": "as needed"
        },
        {
            "display": "Lisinopril",
            "dosage": "20 mg",
            "frequency": "daily"
        }
    ],
    "observations": [
        {
            "display": "Hemoglobin A1c",
            "value": 8.7,
            "unit": "%"
        }
    ]
}
{
    "patient": {
        "id": "5678",
        "name": 

# Evaluation

In [153]:
def extract_structured_grade(client, json_response_str, note):
    grader_template = PromptTemplate.from_template(
        """
            Task: Evaluate the following JSON, extracted from a clinical note for Type 2 Diabete patient,
            for accuracy, completeness, and adherence to FHIR standards and terms.

            Evaluation Criteria:
            1. Correct JSON structure (20%)

            Does the JSON follow the expected schema (below)?
            Are data types correct?
            {{
                "patient": {{
                    "id": "<string>",
                    "name": "<string>",
                    "age": <integer>,
                    "sex": "<string>"
                }},
                "symptoms": ["<string>"],
                "suspected_complications": ["<string>"],
                "comorbidities": ["<string>"],
                "medications": [
                    {{ "display": "<string>", "dosage": "<string>", "frequency": "<string>" }}
                ],
                "observations": [
                    {{ "display": "<string>", "value": <float>, "unit": "<string>" }},
                ]
            }}

            2. Data type validity and Terminology Mapping (30%)

            Are the values and units valid for the given observation?
            Is the display name appropriately mapped? (e.g. A1c -> Hemoglobin A1C)
            Are unit values using UCUM (e.g., "mmHg" for pressure, "beats/min" for heart rate, and "%" for HbA1c)?

            3. Data Completeness & Accuracy (30%)

            Are all expected observations included?
            Are value entries within a reasonable medical range?
            Are T2D complications accurate given the following definitions? Are there inconsistencies between the notes and the definitions below?
            - "Peripheral neuropathy": Indicated by pain, numbness, tingling, loss of feeling in the extremities.
            - "Diabetic nephropathy": Indicated by elevated creatinine levels, protein in urine (albuminuria), swelling in legs and feet, fatigue, high blood pressure.
            - "Diabetic retinopathy": Indicated by blurred vision, floaters, difficulty seeing at night, partial or total vision loss.
            - "Skin conditions": Indicated by dry skin, itching, frequent skin infections, slow wound healing, dark patches on skin (acanthosis nigricans).
            - "Heart and blood vessel disease": Indicated by high blood pressure, chest pain, shortness of breath, irregular heartbeat, swelling in legs, dizziness, increased risk of stroke or heart attack.


            4. Consistency & Errors (20%)

            Are similar entries formatted the same way?
            Are there duplicate or conflicting entries?



            Now, Evaluate the Following JSON....
            {json}

            given the following clinical note:
            {note}

            Output a structured evaluation report in JSON, which is this format exactly:
            {{
                "score": "integer (0-100)",
                "errors_detected": ["List of errors"],
                "suggested_fixes": "JSON with corrected data" (or null if no corrections)
            }}
            """
    )
    prompt = grader_template.format(json=json_response_str, note=note)
    structured_output = extract_json_response(client, prompt)
    return structured_output



In [154]:
 grades = [extract_structured_grade(client, json_response_str, note) for json_response_str, note in zip(structured_data, df["note"])]

In [156]:
for i in range(5):
    print(json.dumps(json.loads(grades[i]), indent=4))


{
    "score": 90,
    "errors_detected": [
        "Missing observations for blood pressure, which is important for a patient with hypertension and diabetes.",
        "The 'dosage' for Insulin is not specific. 'Occasional' is not a valid dosage."
    ],
    "suggested_fixes": {
        "patient": {
            "id": "1234",
            "name": "John Smith",
            "age": 58,
            "sex": "male"
        },
        "symptoms": [
            "fatigue",
            "blurred vision",
            "mild numbness in feet"
        ],
        "suspected_complications": [
            "Peripheral neuropathy",
            "Diabetic retinopathy"
        ],
        "comorbidities": [
            "Type 2 Diabetes",
            "Hypertension"
        ],
        "medications": [
            {
                "display": "Metformin",
                "dosage": "500 mg",
                "frequency": "BID"
            },
            {
                "display": "Insulin",
                "dosage

# Unit Tests (didn't complete out of time)

In [146]:
input = """
Patient: Kevin Wright, 66-year-old male
Medical Record #: 9292
Follow-up for Type 2 Diabetes and chronic back pain. States the pain worsens when he sits in his recliner too long.

Current meds: Metformin 500 mg TID, Ibuprofen PRN for back pain, and Lisinopril 10 mg daily for mild hypertension. Last A1C: 8.9%. He’s annoyed that the pharmacy recently changed his pill bottle labels, making them harder to read.
Complains of slight dizziness upon standing, possibly orthostatic. He also mentions he’s been skipping breakfast, which might contribute to blood sugar swings.

Assessment & Plan: Evaluate orthostatic hypotension. Increase dietary supervision, possibly add short-acting insulin if glycemic control remains inadequate. Suggest physical therapy referral for back pain.
"""

expected_output = """
{
    "patient": {
        "id": "9292",
        "name": "Kevin Wright",
        "age": 66,
        "sex": "male"
    },
    "symptoms": [
        "chronic back pain",
    ],
    "suspected_complications": [],
    "comorbidities": [
        "hypertension",
        "slight dizziness upon standing"
    ],
    "medications": [
        {
            "display": "Metformin",
            "dosage": "500 mg",
            "frequency": "TID"
        },
        {
            "display": "Ibuprofen",
            "dosage": null,
            "frequency": PRN
        },
        {
            "display": "Lisinopril",
            "dosage": "10 mg",
            "frequency": "daily"
        }
    ],
    "observations": [
        {
            "display": "Hemoglobin A1c",
            "value": 8.9,
            "unit": "%"
        }
    ]
}
"""


{
    "patient": {
        "id": "1234",
        "name": "John Smith",
        "age": 58,
        "sex": "male"
    },
    "symptoms": [
        "fatigue",
        "blurred vision",
        "mild numbness in feet"
    ],
    "suspected_complications": [
        "Peripheral neuropathy",
        "Diabetic Retinopathy"
    ],
    "comorbidities": [
        "Type 2 Diabetes",
        "Hypertension"
    ],
    "medications": [
        {
            "display": "Metformin",
            "dosage": "500 mg",
            "frequency": "BID"
        },
        {
            "display": "Insulin",
            "dosage": "occasional",
            "frequency": "as needed"
        },
        {
            "display": "Lisinopril",
            "dosage": "20 mg",
            "frequency": "daily"
        }
    ],
    "observations": [
        {
            "display": "Hemoglobin A1c",
            "value": 8.7,
            "unit": "%"
        }
    ]
}
{
    "patient": {
        "id": "5678",
        "name": 